<a href="https://colab.research.google.com/github/MateuszOszczypala/Data_public/blob/main/Genetic_algorithm_RAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import math
import matplotlib.pyplot as plt
from google.colab import drive
from itertools import combinations
from scipy.spatial.distance import pdist

In [ ]:
# Basic parameters of GA
number_of_generations = 10000
population_size = 200
tournament_size = 5
crossover_probability = 0.5
mutation_probability = 0.2
elitism = 0.04

In [ ]:
# Basic parameters of system
number_of_subsystems = 10
working_failure_rate = [0.8147, 0.9058, 0.1270, 0.9134, 0.6324, 0.0975, 0.2785, 0.5469, 0.9575, 0.9649]
standby_failure_rate = [0.0943, 0.1049, 0.0147, 0.1057, 0.0732, 0.0113, 0.0322, 0.0633, 0.1108, 0.1117]
repair_rate = [2.9412, 2.9143, 1.9708, 2.6006, 1.2838, 1.8435, 2.8315, 2.5844, 2.9190, 2.3115]
warm_standby_switching_rate = [5.3571, 13.4913, 14.3399, 11.7874, 2.5774, 12.4313, 8.9223, 11.5548, 6.7119, 12.0605]
cold_standby_switching_rate = [2.6785, 6.7457, 7.1700, 5.8937, 1.2887, 6.2157, 4.4611, 5.7774, 3.3559, 6.0302]
k = [3, 3, 1, 1, 2, 2, 1, 1, 3, 3]
unit_cost = [5.67, 8.36, 8.57, 4.31, 6.43, 6.12, 7.52, 7.97, 8.28, 4.93]
unit_weight = [5.22, 6.94, 5.32, 5.68, 10.76, 9.86, 7.22, 11.65, 5.24, 8.07]

In [ ]:
# Constraints
weight_constraint = 400
cost_constraint = 350

In [ ]:
def Create_CTMC_model_no_redundancy(n, working_failure_rate, repair_rate): # strategy 0
  # Initialize CTMC matrix
  CTMC = np.zeros((2, 2))
  # State no. 0 is subsystem working state
  # State no. 1 is subsystem failure state

  # Transition rates
  CTMC[0, 1] = n*working_failure_rate # Failure of working component
  CTMC[1, 0] = repair_rate # Repair
  CTMC[0, 0] = -n*working_failure_rate # diagonal
  CTMC[1, 1] = -repair_rate # diagonal
  return CTMC

In [ ]:
def Create_CTMC_model_hot_standby(n, k, working_failure_rate, repair_rate): # strategy 1
  State_space = np.empty((n-k+2, 3), dtype=int)
  # States no. from 0 to n-k are subsystem working states
  # State no. n-k+1 is subsystem failure state

  # Working states with k working components and remaining in hot standby or failed
  j = 0
  for i in range(n-k+1):
    State_space[i, 0] = k # number of working components
    State_space[i, 1] = n-k-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1
  # Failure states with k-1 operational components and n-k+1 failed components
    State_space[n-k+1, 0] = k-1 # number of working components
    State_space[n-k+1, 1] = 0 # number of hot standby components
    State_space[n-k+1, 2] = n-k+1 # number of failed components

  # Initialize CTMC matrix
  CTMC = np.zeros((n-k+2, n-k+2))

  # Transition rates
  for i in range(n-k+2):
    for j in range(n-k+2):
      if State_space[i, 0] - State_space[j, 0] == 1: # Failure of working component
        CTMC[i, j] = State_space[i, 0] * working_failure_rate
      elif State_space[i, 2] - State_space[j, 2] == 1: # Repair
        CTMC[i, j] = State_space[i, 2] * repair_rate

    # Diagonal elements
  for i in range(n-k+2):
    CTMC[i, i] = -np.sum(CTMC[i, :])

  return CTMC

In [ ]:
def Create_CTMC_model_warm_standby(n, k, working_failure_rate, standby_failure_rate, repair_rate, switching_rate): # strategy 2
  State_space = np.empty((2*n-2*k+3, 3), dtype=int)
  # States no. from 0 to n-k are subsystem working states
  # States no. form n-k+1 to 2n-2k+2 are subsystem failure states

  # Working states with k working components and remaining in warm standby or failed
  j = 0
  for i in range(n-k+1):
    State_space[i, 0] = k # number of working components
    State_space[i, 1] = n-k-j # number of warm standby components
    State_space[i, 2] = j # number of failed components
    j += 1
  # Failure states with k-1 operational components and n-k+1 failed components
  j = 0
  for i in range(n-k+1, 2*n-2*k+3):
    State_space[i, 0] = k-1 # number of working components
    State_space[i, 1] = n-k+1-j # number of warm standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  # Initialize CTMC matrix
  CTMC = np.zeros((2*n-2*k+3, 2*n-2*k+3))

  # Transition rates
  for i in range(2*n-2*k+3):
    for j in range(2*n-2*k+3):
      if State_space[i, 0] - State_space[j, 0] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of working component
        CTMC[i, j] = State_space[i, 0] * working_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of warm standby component
        if State_space[i, 0] == k - 1:
          # CTMC[i, j] = (State_space[i, 1] + State_space[i, 0]) * standby_failure_rate
          CTMC[i, j] = 0 # according to the assumption of non-failing components in subsystem failure state
        else:
          CTMC[i, j] = State_space[i, 1] * standby_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 0] - State_space[i, 0] == 1:  # Switch
        CTMC[i, j] = switching_rate
      elif State_space[i, 2] - State_space[j, 2] == 1 and State_space[j, 1] - State_space[i, 1] == 1:  # Repair
        CTMC[i, j] = State_space[i, 2] * repair_rate

    # Diagonal elements
  for i in range(2*n-2*k+3):
    CTMC[i, i] = -np.sum(CTMC[i, :])

  return CTMC

In [ ]:
def Create_CTMC_model_cold_standby(n, k, working_failure_rate, repair_rate, switching_rate): # strategy 3
  State_space = np.empty((2*n-2*k+3, 3), dtype=int)
  # States no. from 0 to n-k are subsystem working states
  # States no. form n-k+1 to 2n-2k+2 are subsystem failure states

  # Working states with k working components and remaining in cold standby or failed
  j = 0
  for i in range(n-k+1):
    State_space[i, 0] = k # number of working components
    State_space[i, 1] = n-k-j # number of cold standby components
    State_space[i, 2] = j # number of failed components
    j += 1
  # Failure states with k-1 operational components and n-k+1 failed components
  j = 0
  for i in range(n-k+1, 2*n-2*k+3):
    State_space[i, 0] = k-1 # number of working components
    State_space[i, 1] = n-k+1-j # number of cold standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  # Initialize CTMC matrix
  CTMC = np.zeros((2*n-2*k+3, 2*n-2*k+3))

  # Transition rates
  for i in range(2*n-2*k+3):
    for j in range(2*n-2*k+3):
      if State_space[i, 0] - State_space[j, 0] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of working component
        CTMC[i, j] = State_space[i, 0] * working_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 0] - State_space[i, 0] == 1:  # Switch
        CTMC[i, j] = switching_rate
      elif State_space[i, 2] - State_space[j, 2] == 1 and State_space[j, 1] - State_space[i, 1] == 1:  # Repair
        CTMC[i, j] = State_space[i, 2] * repair_rate

    # Diagonal elements
  for i in range(2*n-2*k+3):
    CTMC[i, i] = -np.sum(CTMC[i, :])

  return CTMC

In [ ]:
def Create_CTMC_model_mixed_strategy_active_warm(n, k, working_failure_rate, standby_failure_rate, repair_rate, switching_rate): # strategy 4
  State_space = np.empty((3*n-3*k+3, 3), dtype=int)

  j = 0
  # Working states with k+1 active components
  for i in range(n-k):
    State_space[i, 0] = k+1 # number of working components
    State_space[i, 1] = n-k-1-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1
  j = 0
  # Working states with k active components
  for i in range(n-k, 2*n-2*k+1):
    State_space[i, 0] = k # number of working components
    State_space[i, 1] = n-k-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  j = 0
    # Failed states with k-1 active components
  for i in range(2*n-2*k+1, 3*n-3*k+3):
    State_space[i, 0] = k-1 # number of working components
    State_space[i, 1] = n-k+1-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  # Initialize CTMC matrix
  CTMC = np.zeros((3*n-3*k+3, 3*n-3*k+3))

  # Transition rates
  for i in range(3*n-3*k+3):
    for j in range(3*n-3*k+3):
      if State_space[i, 0] - State_space[j, 0] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of active component
        CTMC[i, j] = State_space[i, 0] * working_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of warm standby component
        if State_space[i, 0] == k - 1:
          # CTMC[i, j] = (State_space[i, 1] + State_space[i, 0]) * standby_failure_rate
          CTMC[i, j] = 0 # according to the assumption of non-failing components in subsystem failure state
        else:
          CTMC[i, j] = State_space[i, 1] * standby_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 0] - State_space[i, 0] == 1:  # Switch
        CTMC[i, j] = (min(min(State_space[i, 1], 2), k + 1 - State_space[i, 0])) * switching_rate
      elif State_space[i, 2] - State_space[j, 2] == 1 and State_space[j, 1] - State_space[i, 1] == 1:  # Repair
        CTMC[i, j] = State_space[i, 2] * repair_rate

    # Diagonal elements
  for i in range(3*n-3*k+3):
    CTMC[i, i] = -np.sum(CTMC[i, :])

  return CTMC

In [ ]:
def Create_CTMC_model_mixed_strategy_active_cold(n, k, working_failure_rate, repair_rate, switching_rate): # strategy 5
  State_space = np.empty((3*n-3*k+3, 3), dtype=int)

  j = 0
  # Working states with k+1 active components
  for i in range(n-k):
    State_space[i, 0] = k+1 # number of working components
    State_space[i, 1] = n-k-1-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1
  j = 0
  # Working states with k active components
  for i in range(n-k, 2*n-2*k+1):
    State_space[i, 0] = k # number of working components
    State_space[i, 1] = n-k-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  j = 0
    # Failed states with k-1 active components
  for i in range(2*n-2*k+1, 3*n-3*k+3):
    State_space[i, 0] = k-1 # number of working components
    State_space[i, 1] = n-k+1-j # number of hot standby components
    State_space[i, 2] = j # number of failed components
    j += 1

  # Initialize CTMC matrix
  CTMC = np.zeros((3*n-3*k+3, 3*n-3*k+3))

  # Transition rates
  for i in range(3*n-3*k+3):
    for j in range(3*n-3*k+3):
      if State_space[i, 0] - State_space[j, 0] == 1 and State_space[j, 2] - State_space[i, 2] == 1:  # Failure of active component
        CTMC[i, j] = State_space[i, 0] * working_failure_rate
      elif State_space[i, 1] - State_space[j, 1] == 1 and State_space[j, 0] - State_space[i, 0] == 1:  # Switch
        CTMC[i, j] = (min(min(State_space[i, 1], 2), k + 1 - State_space[i, 0])) * switching_rate
      elif State_space[i, 2] - State_space[j, 2] == 1 and State_space[j, 1] - State_space[i, 1] == 1:  # Repair
        CTMC[i, j] = State_space[i, 2] * repair_rate

    # Diagonal elements
  for i in range(3*n-3*k+3):
    CTMC[i, i] = -np.sum(CTMC[i, :])

  return CTMC

In [ ]:
def Ergodic_probabilities(CTMC):
  CTMC_transposed = CTMC.T
  CTMC_transposed[0, :CTMC_transposed.shape[0]] = 1
  P = np.zeros((CTMC_transposed.shape[0], 1))
  P[0, 0] = 1
  Ergodic_prob = np.linalg.inv(CTMC_transposed) @ P
  return Ergodic_prob

In [ ]:
def Availability_of_subsystems(n, k, Ergodic_prob, redundancy_strategy):

  if redundancy_strategy == 0: # no redundancy
    Subsystem_availabilities = np.sum(Ergodic_prob[0:1])

  elif redundancy_strategy == 1: # hot standby
    Subsystem_availabilities = np.sum(Ergodic_prob[0:n-k+1])

  elif redundancy_strategy == 2: # warm standby
    Subsystem_availabilities = np.sum(Ergodic_prob[0:n-k+1])

  elif redundancy_strategy == 3: # cold standby
    Subsystem_availabilities = np.sum(Ergodic_prob[0:n-k+1])

  elif redundancy_strategy == 4: # mixed strategy(active + warm)
    Subsystem_availabilities = np.sum(Ergodic_prob[0:2*n-2*k+1])

  elif redundancy_strategy == 5: # mixed strategy(active + cold)
    Subsystem_availabilities = np.sum(Ergodic_prob[0:2*n-2*k+1])

  return Subsystem_availabilities

In [ ]:
def objective_function(Subsystem_availabilities):
  A = Subsystem_availabilities
  U = 1 - A
  System_availability = (
        A[0]*A[1]*A[2]*A[3]+
        A[0]*A[1]*A[5]*A[9]*(U[2]+A[2]*U[3])+
        A[0]*A[4]*A[8]*A[9]*(U[1]+A[1]*U[2]*U[5]+A[1]*A[2]*U[3]*U[5])+
        A[6]*A[7]*A[8]*A[9]*(U[0]+A[0]*U[1]*U[4]+A[0]*A[1]*U[2]*U[4]*U[5]+A[0]*A[1]*A[2]*U[3]*U[4]*U[5])+
        A[1]*A[2]*A[3]*A[4]*A[6]*A[7]*U[0]*(U[8]+A[8]*U[9])+
        U[0]*A[2]*A[3]*A[5]*A[6]*A[7]*A[8]*U[9]*(U[1]+A[1]*U[4])+
        A[0]*U[1]*A[2]*A[3]*A[5]*A[6]*A[7]*A[8]*A[9]*U[9]+
        A[0]*U[1]*A[2]*A[3]*A[4]*A[5]*A[8]*U[9]*(U[6]+A[6]*U[7])+
        U[0]*A[1]*A[4]*A[5]*A[6]*A[7]*A[8]*U[8]*A[9]*(U[2]+A[2]*U[3])
    )
  return System_availability

In [ ]:
def tournament_selection(population, objective_function_value, tournament_size, population_size, elitism):
  parent = []
  individual_index = np.zeros(tournament_size, dtype=int)
  for j in range(tournament_size):
    individual_index[j] = np.random.randint(0, population_size)
  parent_objective_value = max(objective_function_value[individual_index])
  for j in range(tournament_size):
    if objective_function_value[individual_index[j]] == parent_objective_value:
      parent = (population[:,:,individual_index[j]]) # tournament winner
  return parent

In [ ]:
def single_point_crossover(parent1, parent2):
  point = random.randint(1, parent1.shape[0]-1)

  offspring1 = np.empty((8, number_of_subsystems, 1, 1), dtype=int)
  offspring2 = np.empty((8, number_of_subsystems, 1, 1), dtype=int)
  offspring1 = parent1.copy()
  offspring2 = parent2.copy()
  offspring1[:point, :] = parent2[:point, :]
  offspring2[:point, :] = parent1[:point, :]
  offspring1 = np.clip(offspring1, 0, 1)
  offspring2 = np.clip(offspring2, 0, 1)
  return offspring1, offspring2

In [ ]:
def two_point_crossover(parent1, parent2):
  point1 = random.randint(1, parent1.shape[0]-2)
  point2 = random.randint(point1+1, parent1.shape[0]-1)
  offspring1 = np.empty((8, number_of_subsystems, 1, 1), dtype=int)
  offspring2 = np.empty((8, number_of_subsystems, 1, 1), dtype=int)
  offspring1 = parent1.copy()
  offspring2 = parent2.copy()
  offspring1[point1:point2, :] = parent2[point1:point2, :]
  offspring2[point1:point2, :] = parent1[point1:point2, :]
  offspring1 = np.clip(offspring1, 0, 1)
  offspring2 = np.clip(offspring2, 0, 1)
  return offspring1, offspring2

In [ ]:
def mutation(offspring, mutation_probability):
  mutated_offspring = np.empty((offspring.shape[0], offspring.shape[1], 1, 1), dtype=int)
  for j in range(offspring.shape[1]):
    for i in range(offspring.shape[0]):
      if random.random() < mutation_probability:
        mutated_offspring[i,j,0,0] = 1 - offspring[i,j]
      else:
        mutated_offspring[i,j,0,0] = offspring[i,j]
  mutated_offspring = np.clip(mutated_offspring, 0, 1)
  return mutated_offspring[:,:,0,0]
  return mutated_offspring[:,:,0,0]

In [ ]:
def decode_chromosomes(chromosomes):
  priority = np.empty((chromosomes.shape[1]), dtype=int)
  for i in range(chromosomes.shape[1]):
    bits = chromosomes[:chromosomes.shape[0], i]
    bits = np.clip(bits, 0, 1)  # Clip values to 0 or 1
    # Convert the bits to a string, then convert the binary string to a decimal integer.
    priority[i] = int(''.join(map(str, bits)), 2)
  return priority

In [ ]:
def component_allocation(redundancy_strategy, priority, weight_available, unit_weight, cost_available, unit_cost):
  a = np.zeros(priority.shape[0])
  for i in range(priority.shape[0]):
    a[i] = priority[i]/255
  standby_components = np.zeros(priority.shape[0], dtype=int)

  for i in range(priority.shape[0]):
    if redundancy_strategy[i] == 0: # no redundancy
      standby_components[i] = 0
    else:
      standby_components[i] = round(a[i] * min(round(weight_available/unit_weight[i]),round(cost_available/unit_cost[i])))
  return standby_components

In [ ]:
def identify_redundacy_strategy(strategy_bits):
  redundancy_strategy = []
  if np.array_equal(strategy_bits, [0, 0, 0]):
    redundancy_strategy = 0 # no redundancy
  elif np.array_equal(strategy_bits, [1, 0, 0]):
    redundancy_strategy = 1 #hot standby
  elif np.array_equal(strategy_bits, [0, 1, 0]):
    redundancy_strategy = 2 #warm standby
  elif np.array_equal(strategy_bits, [0, 0, 1]):
    redundancy_strategy = 3 #cold standby
  elif np.array_equal(strategy_bits, [1, 1, 0]):
    redundancy_strategy = 4 #mixed 1 active + warm standby
  elif np.array_equal(strategy_bits, [1, 0, 1]):
    redundancy_strategy = 5 #mixed 1 active + cold standby

  return redundancy_strategy

In [ ]:
# Initialization
def initialization(number_of_subsystems, population_size):
  initial_population = np.random.randint(2, size=(11, number_of_subsystems, population_size))
  initial_population = np.clip(initial_population, 0, 1)

# Initialization
def initialization(number_of_subsystems, population_size):
  initial_population = np.random.randint(2, size=(11, number_of_subsystems, population_size))
  initial_population = np.clip(initial_population, 0, 1)

  # check if the redundancy strategy bits encode a feasible redundacy strategy
  for i in range(population_size):
    for j in range(number_of_subsystems):
      while (initial_population[9, j, i] == 1) and (initial_population[10, j, i] == 1):
        initial_population[8:11, j, i] = np.random.randint(2, size=3)

  return initial_population

  return initial_population

In [ ]:
# Genetic algorithm
population = np.empty((11, number_of_subsystems, population_size, number_of_generations), dtype=int)
population[:,:,:,0] = initialization(number_of_subsystems, population_size)
elitist_population = np.empty((11, number_of_subsystems, round(population_size*elitism), number_of_generations), dtype=int)
parents = np.empty((11, number_of_subsystems, population_size-round(population_size*elitism), number_of_generations), dtype=int)
offsprings = np.empty((11, number_of_subsystems, population_size-round(population_size*elitism), number_of_generations), dtype=int)
objective_function_value = np.full((population_size, number_of_generations), np.nan)
n = np.full((number_of_subsystems, population_size, number_of_generations), np.nan)
redundancy_strategy = np.full((number_of_subsystems, population_size, number_of_generations), np.nan)
priorities = np.full((number_of_subsystems, population_size, number_of_generations), np.nan)

available_weight = 0
for i in range(number_of_subsystems):
  available_weight += k[i]*unit_weight[i]


available_cost = 0
for i in range(number_of_subsystems):
  available_cost += k[i]*unit_cost[i]


for num_it in range(number_of_generations):
  for num_individual in range(population_size):

    # decode chromosomes of population into priorities of subsystems
    priority = decode_chromosomes(population[0:7,:,num_individual,num_it])
    priorities[:,num_individual, num_it] = priority

    # identify the redundancy strategy
    for i in range(number_of_subsystems):
      strategy_bits = population[8:11, i, num_individual, num_it]
      redundancy_strategy[i, num_individual, num_it] = identify_redundacy_strategy(strategy_bits)

    # allocate standby components to subsystems
    standby_components = component_allocation(redundancy_strategy[:, num_individual, num_it], priority, available_weight, unit_weight, available_cost, unit_cost)
    n_components = np.zeros(number_of_subsystems, dtype=int)
    for i in range(number_of_subsystems):
      n[i, num_individual, num_it] = k[i] + int(standby_components[i])
      n_components[i] = k[i] + int(standby_components[i])

    # calculate subsystem availabilities
    Subsystem_availabilities = np.full((number_of_subsystems), np.nan)

    for num_subsystem in range(number_of_subsystems):
      if redundancy_strategy[num_subsystem, num_individual, num_it] == 0:
        CTMC = Create_CTMC_model_no_redundancy(n_components[num_subsystem], working_failure_rate[num_subsystem], repair_rate[num_subsystem])

      elif redundancy_strategy[num_subsystem, num_individual, num_it] == 1:
        CTMC = Create_CTMC_model_hot_standby(n_components[num_subsystem], k[num_subsystem], working_failure_rate[num_subsystem], repair_rate[num_subsystem])

      elif redundancy_strategy[num_subsystem, num_individual, num_it] == 2:
        CTMC = Create_CTMC_model_warm_standby(n_components[num_subsystem], k[num_subsystem], working_failure_rate[num_subsystem], standby_failure_rate[num_subsystem], repair_rate[num_subsystem], warm_standby_switching_rate[num_subsystem])

      elif redundancy_strategy[num_subsystem, num_individual, num_it] == 3:
        CTMC = Create_CTMC_model_cold_standby(n_components[num_subsystem], k[num_subsystem], working_failure_rate[num_subsystem], repair_rate[num_subsystem], cold_standby_switching_rate[num_subsystem])

      elif redundancy_strategy[num_subsystem, num_individual, num_it] == 4:
        CTMC = Create_CTMC_model_mixed_strategy_active_warm(n_components[num_subsystem], k[num_subsystem], working_failure_rate[num_subsystem], standby_failure_rate[num_subsystem], repair_rate[num_subsystem], warm_standby_switching_rate[num_subsystem])

      elif redundancy_strategy[num_subsystem, num_individual, num_it] == 5:
        CTMC = Create_CTMC_model_mixed_strategy_active_cold(n_components[num_subsystem], k[num_subsystem], working_failure_rate[num_subsystem], repair_rate[num_subsystem], cold_standby_switching_rate[num_subsystem])

      Ergodic_prob = Ergodic_probabilities(CTMC)
      Subsystem_availabilities[num_subsystem] = Availability_of_subsystems(n_components[num_subsystem], k[num_subsystem], Ergodic_prob, redundancy_strategy[num_subsystem, num_individual, num_it])

      del CTMC, Ergodic_prob

    # objective value (system availability) calculation
    if np.sum(n[:, num_individual, num_it]*unit_cost) > cost_constraint or np.sum(n[:, num_individual, num_it]*unit_weight) > weight_constraint:
      num_components = np.full((number_of_subsystems), np.nan)
      for i in range(number_of_subsystems):
        num_components[i] = n[i, num_individual, num_it]
        system_cost = 0
      for i in range(number_of_subsystems):
        system_cost = system_cost + num_components[i]*unit_cost[i]
        system_weight = 0
      for i in range(number_of_subsystems):
        system_weight = system_weight + num_components[i]*unit_weight[i]
      if system_cost/cost_constraint > system_weight/weight_constraint:
        penalty_function = 0.01**(system_cost/cost_constraint)
      else:
        penalty_function = 0.01**(system_weight/weight_constraint)
      objective_function_value[num_individual, num_it] = objective_function(Subsystem_availabilities)*penalty_function
    else:
      objective_function_value[num_individual, num_it] = objective_function(Subsystem_availabilities)

  # elitism
  sorted_individuals_indices = np.argsort(objective_function_value[:,num_it])[::-1]
  for i in range(round(elitism*population_size)):
    for j in range(population_size):
      if sorted_individuals_indices[i] == j:
        elitist_population[:,:,i,num_it] = population[:,:,j,num_it]
  if num_it < number_of_generations - 1:
    for i in range(round(elitism*population_size)):
      population[:,:,i, num_it+1] = elitist_population[:,:,i,num_it]

  # selection
  for i in range(round(population_size-elitism*population_size)):
    parents[:,:,i,num_it] = tournament_selection(population[:, :, :, num_it], objective_function_value[:, num_it], tournament_size, population_size, elitism)

  # crossover for parents to create offsprings
  if num_it < number_of_generations - 1:
    for i in range(0, parents.shape[2]-1, 2):
      if random.random() < crossover_probability:
        offspring1, offspring2 = single_point_crossover(parents[:,:,i,num_it], parents[:,:,i+1,num_it])
      else:
        offspring1, offspring2 = two_point_crossover(parents[:,:,i,num_it], parents[:,:,i+1,num_it])

      offsprings[:,:,i,num_it] = offspring1[:,:]
      offsprings[:,:,i+1,num_it] = offspring2[:,:]
    if parents.shape[2] % 2 == 1:
      offsprings[:,:,i,num_it] = parents[:,:,i,num_it]

  # mutation
  adaptive_mutation_probability = mutation_probability*(((number_of_generations-num_it)/number_of_generations)**2)
  for i in range(offsprings.shape[2]):
    offsprings[:,:,i,num_it] = mutation(offsprings[:,:,i,num_it], adaptive_mutation_probability)

  # check if the redundancy strategy bits encode a feasible redundacy strategy
  for i in range(offsprings.shape[2]):
    for j in range(number_of_subsystems):
      while (offsprings[9, j, i, num_it] == 1) and (offsprings[10, j, i, num_it] == 1):
        random_redundancy_strategy_bit = np.random.randint(8, 11)
        offsprings[random_redundancy_strategy_bit, j, i, num_it] = np.random.randint(0, 2)

  # concatenate the elitist individuals and mutated offsprings
  if num_it < number_of_generations - 1:
    population[:,:,round(elitism*population_size):,num_it+1] = offsprings[:,:,:,num_it]


In [ ]:
print(np.nanmax(objective_function_value))
print(np.unravel_index(np.argmax(objective_function_value), objective_function_value.shape))
print(redundancy_strategy[:,0,83])
print(n[:,0,83])
print(k)

In [ ]:
print((objective_function_value[:,99]))

In [ ]:
plt.plot(np.max(objective_function_value, axis=0))
plt.xlabel('Number of generation')
plt.ylabel('System availability')
plt.show()

In [ ]:
num_subsystems, num_points, num_generations = priorities.shape

sum_distances = np.zeros(num_generations)

for z in range(num_generations):
    total_distance = 0

    points = priorities[:, :, z].T  # (y, x) - points in the space

    # Distances among points in solution space in each generation
    total_distance = np.sum(pdist(points))  # pdist calculates distances between each two points

    sum_distances[z] = total_distance

plt.plot(sum_distances)
plt.xlabel('Number of generation')
plt.ylabel('Sum of distances between points')
plt.show()

In [ ]:
def hamming_distance(individual1, individual2):
    return np.sum(individual1 != individual2)

def calculate_total_hamming_distance(population):

    total_hamming_distance = 0

    for i in range(population_size):
        for j in range(i + 1, population_size):
            # Hamming distance for each individual pair
            total_hamming_distance += np.sum([hamming_distance(population[:,:, i], population[:,:, j])])

    return total_hamming_distance

total_hamming_distances = []

# Sum of Hamming distance for each generation
for generation_idx in range(number_of_generations):
    total_hamming_distance = calculate_total_hamming_distance(population[:,:,:,generation_idx])
    total_hamming_distances.append(total_hamming_distance)

average_hamming_distances = np.empty(number_of_generations)
for i in range(number_of_generations):
    average_hamming_distances[i] = np.array(total_hamming_distances[i]) / (population_size * (population_size - 1) / 2)

# Plot
import matplotlib.pyplot as plt
plt.plot(range(1, number_of_generations + 1), average_hamming_distances, marker='o', markersize=2)
plt.xlabel('Generation')
plt.ylabel('The average Hamming distance')
plt.grid(True)
plt.show()